In [1]:
import pandas as pd

In [3]:
car=pd.read_csv('quikr_car.csv')

In [4]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing XO eRLX Euro III,Hyundai,2007,"80,000","45,000 kms",Petrol
1,Mahindra Jeep CL550 MDI,Mahindra,2006,"4,25,000",40 kms,Diesel
2,Maruti Suzuki Alto 800 Vxi,Maruti,2018,Ask For Price,"22,000 kms",Petrol
3,Hyundai Grand i10 Magna 1.2 Kappa VTVT,Hyundai,2014,"3,25,000","28,000 kms",Petrol
4,Ford EcoSport Titanium 1.5L TDCi,Ford,2014,"5,75,000","36,000 kms",Diesel


In [5]:
#year has non year values

In [ ]:
#year object is int
#price has ask for price
#price object to int
#kms_driver has kms with integers
#kms_driven object to int
#kms_driven has nan values
#fules type has nan values
#keep first 3 words of name


cleaning


In [6]:
backup=car.copy()

In [8]:
car=car[car['year'].str.isnumeric()]

In [9]:
car['year']=car['year'].astype(int)

In [15]:
car=car[car['Price']!='Ask For Price']

In [16]:
car['Price']=car['Price'].str.replace(',','').astype(int)

<ipython-input-16-7c631ea95249>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car['Price']=car['Price'].str.replace(',','').astype(int)


In [18]:
car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

<ipython-input-18-311c68d9709d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')


In [19]:
car['kms_driven']

0       45000
1          40
3       28000
4       36000
6       41000
        ...  
886    132000
888     27000
889     40000
890    Petrol
891    Petrol
Name: kms_driven, Length: 819, dtype: object

In [20]:
car=car[car['kms_driven'].str.isnumeric()]

In [21]:
car['kms_driven']=car['kms_driven'].astype(int)

In [22]:
car['kms_driven']

0       45000
1          40
3       28000
4       36000
6       41000
        ...  
883     50000
885     30000
886    132000
888     27000
889     40000
Name: kms_driven, Length: 817, dtype: int64

In [25]:
car = car[~car['fuel_type'].isna()]

In [26]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [28]:
car.reset_index(drop=True,inplace=True)

In [29]:
car=car[car['Price']<6e6].reset_index(drop=True)

In [30]:
car.to_csv('cleaned_car.csv')

model


In [31]:
x=car.drop(columns='Price')
y=car['Price']

In [32]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [35]:
ohe=OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder()

In [43]:
column_trans = make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_), ['name', 'company', 'fuel_type']),
    remainder='passthrough'
)

In [37]:
lr=LinearRegression()

In [39]:
pipe = make_pipeline(column_trans, lr) # Pass lr as a separate argument

In [41]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [46]:
column_trans = make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_, handle_unknown='ignore'), ['name', 'company', 'fuel_type']),
    remainder='passthrough'
)

In [49]:
# Define column_trans before creating the pipeline
column_trans = make_column_transformer(
    (OneHotEncoder(handle_unknown='ignore'), ['name', 'company', 'fuel_type']),
    remainder='passthrough'
)

# Now create the pipeline
pipe = make_pipeline(column_trans, lr) # Pass lr as a separate argument

# Fit the pipeline
pipe.fit(x_train,y_train)

# Make predictions
y_pred=pipe.predict(x_test)

In [50]:
y_pred

array([ 4.24441914e+05,  2.43870310e+05,  3.63706823e+05,  5.28884264e+05,
        2.78606228e+05,  3.35626791e+05,  2.27267698e+05,  4.86815086e+05,
        2.73051935e+05,  2.62838690e+05, -5.21446860e+04,  3.80760102e+05,
        6.04093873e+05,  9.48401833e+04,  4.36722446e+05,  2.39097865e+05,
        2.70259952e+05,  2.52510167e+05,  6.04785875e+05,  7.78537625e+05,
        2.67427745e+05,  4.80099618e+05,  8.72874831e+05,  1.66117394e+05,
        7.02681750e+05,  3.73839209e+05,  4.18449220e+05,  9.90328595e+05,
        6.26215686e+05,  2.40920611e+05,  4.30364410e+05,  3.03537183e+05,
        2.86548280e+05,  2.61744788e+05,  3.60626782e+05,  6.42992164e+05,
        5.25960906e+05,  2.19771231e+06,  3.57516696e+05,  6.33608260e+05,
        3.47619872e+05,  3.19329341e+05,  2.69452277e+05, -7.66188612e+04,
        4.25455620e+05,  1.37644841e+05,  4.53916015e+05,  5.21199310e+05,
        3.42015467e+05,  1.39652626e+06,  1.34533248e+05,  4.18308918e+05,
        2.37015374e+05,  

In [52]:
!pip install scikit-learn
from sklearn.metrics import r2_score

# your existing code to calculate y_pred goes here

r2_score(y_test,y_pred)

0.6733138763306215

In [53]:
score=[]
for i in range(1000):
  x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=i)
  lr=LinearRegression()
  pipe=make_pipeline(column_trans,lr)
  pipe.fit(x_train,y_train)
  y_pred=pipe.predict(x_test)
  score.append(r2_score(y_test,y_pred))

In [54]:
import numpy as np

In [56]:
score[np.argmax(score)]

0.8456515104686065

In [59]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=np.argmax(score))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(x_train,y_train)
y_pred=pipe.predict(x_test)
r2_score(y_test,y_pred)

0.8456515104686065

In [60]:
import pickle

In [61]:
pickle.dump(pipe,open('LinearRegressionModel.pkl','wb'))

In [62]:
pipe.predict(pd.DataFrame([['Maruti Suzki Swift', 'Maruti',2019,100,'Petrol']],columns=['name','company','year','kms_driven','fuel_type']))

array([561973.36662702])